In [ ]:
import datasets
data = datasets.load_dataset('squad_v2')
train_dataset, test_dataset = data['train'], data['validation']

In [ ]:
import datasets
import numpy as np
import pandas as pd
import random
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, DataCollatorForSeq2Seq
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")

In [ ]:
special_tokens_dict = {'additional_special_tokens': ['<|title|>', '<|question|>', '<|answer|>', '<|context|>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
import tqdm
new_train, new_test = {'context': [], 'questions': [], 'answers': [], 'title': []}, {'context': [], 'questions': [], 'answers': [], 'title': []}

contexts, questions, answers, titles = train_dataset['context'], train_dataset['question'], train_dataset['answers'], train_dataset['title']
for i in tqdm.tqdm(range(len(train_dataset['id']))):
    if len(answers[i]['text']) == 0:
        continue

    context, question, answer, title = contexts[i], questions[i], answers[i]['text'][0], titles[i]

    if title.lower() not in question.lower() or title.lower() in answer.lower():
       continue

    if len(new_train['context']) > 0 and new_train['context'][-1] == context:
        new_train['questions'][-1].append(question)
        new_train['answers'][-1].append(answer)
    else:
        new_train['questions'].append([question])
        new_train['answers'].append([answer])
        new_train['title'].append(title)
        new_train['context'].append(context)

contexts, questions, answers, titles = test_dataset['context'], test_dataset['question'], test_dataset['answers'], test_dataset['title']
for i in tqdm.tqdm(range(len(test_dataset['id']))):
    if len(answers[i]['text']) == 0:
        continue

    context, question, answer, title = contexts[i], questions[i], answers[i]['text'][0], titles[i]

    if len(new_test['context']) > 0 and new_test['context'][-1] == context:
        new_test['questions'][-1].append(question)
        new_test['answers'][-1].append(answer)
    else:
        new_test['questions'].append([question])
        new_test['answers'].append([answer])
        new_test['title'].append(title)
        new_test['context'].append(context)

In [ ]:
train_ds, test_ds = datasets.Dataset.from_dict(new_train), datasets.Dataset.from_dict(new_test)

In [ ]:
max_input_length = 512
max_target_length = 512
def preprocess_function(examples):
    inputs = ["<|title|> " + examples['title'][i] + " <|context|> " + examples['context'][i] for i in range(len(examples["context"]))]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer([' '.join(["<|answer|> " + examples["answers"][j][i] + " <|question|> " + examples['questions'][j][i] for i in range(len(examples['questions'][j]))]) for j in range(len(examples['questions']))], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
import evaluate
rouge = evaluate.load("rouge")
meteor = evaluate.load('meteor')
bleu = evaluate.load('bleu')

import nltk
import numpy as np
nltk.download('punkt')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    rouge_output2 = rouge.compute(predictions=decoded_preds, references=decoded_labels, rouge_types=["rouge2"])["rouge2"]
    rouge_output1 = rouge.compute(predictions=decoded_preds, references=decoded_labels, rouge_types=["rouge1"])["rouge1"]
    bleu_vals = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_vals = meteor.compute(predictions=decoded_preds, references=decoded_labels)

    return {
        "rouge1": rouge_output1,
        "rouge2": rouge_output2,
        "meteor": meteor_vals['meteor'],
        "blue": bleu_vals['bleu']
    }

In [ ]:
train_dataset_tok = train_ds.map(preprocess_function, batched=True)
test_dataset_tok = test_ds.map(preprocess_function, batched=True)

In [ ]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    f"question-generator",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_strategy='no',
    num_train_epochs=5,
    warmup_steps=500,
    predict_with_generate=True,
    gradient_accumulation_steps=8,
    logging_steps=100
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset_tok,
    eval_dataset=test_dataset_tok,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
out_dir = ''
trainer.push_to_hub(out_dir)